In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#  **Scenario 2: Overload Simulation **

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define a function for demand response based on price
def demand_response(price):
    """Estimate demand based on price."""
    # Assume a linear demand function: as price increases, demand decreases
    return max(100 - (price * 50), 0)  # Adjust the parameters based on your needs

# Overload simulation with Stackelberg Pricing Model
def overload_simulation_with_pricing(station_data):
    """
    Simulate high demand at a few stations and adjust prices accordingly.
    
    Parameters:
    station_data (DataFrame): The DataFrame containing charging station data.
    
    Returns:
    DataFrame: Updated station data with new prices and demand.
    """
    overloaded_stations = station_data.sample(frac=0.1)  # Randomly select 10% of stations
    overload_increase = 15  # Increase kWh total by 15 for overloaded stations
    
    # Increase the kWh total for overloaded stations
    for index, station in overloaded_stations.iterrows():
        station_data.at[index, 'kwhTotal'] += overload_increase  # Simulate increased demand

    # Update prices and calculate new demand based on overload
    for index, station in overloaded_stations.iterrows():
        # Increase price based on overload situation
        new_price = station_data.at[index, 'dollars'] * (1 + (overload_increase / 100))  # Increase by a certain percentage
        station_data.at[index, 'dollars'] = new_price
        # Update demand based on new price
        station_data.at[index, 'new_demand'] = demand_response(new_price)

    return station_data

# Sample station data for testing
# Create a sample DataFrame with random data
data = {
    'sessionId': range(1, 101),
    'kwhTotal': np.random.uniform(0, 10, 100),  # Random kWh totals
    'dollars': np.random.uniform(0.5, 1.5, 100)  # Random prices
}
station_data = pd.DataFrame(data)

# Run the overload simulation with pricing
updated_station_data = overload_simulation_with_pricing(station_data)

# Display updated data for verification
print(updated_station_data.head())

# Function to plot the prices and demands
def plot_price_vs_demand(station_data):
    """Plot the prices and demands of charging stations."""
    plt.figure(figsize=(12, 6))
    plt.plot(station_data['sessionId'], station_data['dollars'], label='Price ($)', color='blue', marker='o')
    plt.plot(station_data['sessionId'], station_data['new_demand'], label='New Demand', color='orange', marker='x')
    plt.title('Price and Demand of Charging Stations')
    plt.xlabel('Session ID')
    plt.ylabel('Value')
    plt.legend()
    plt.grid()
    plt.show()

# Call the plotting function to visualize the results
plot_price_vs_demand(updated_station_data)


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
def demand_response(price):
    """Estimate demand based on price using a smoother demand curve with minimum demand."""
    # Ensure the price is a valid number and return demand based on the price
    if price > 0:
        # Using a logarithmic decrease, with a minimum demand threshold of 5
        return max(5, 100 - np.log(price + 1) * 15)
    else:
        return 0  # If the price is invalid or zero, return zero demand


# Overload simulation with Stackelberg Pricing Model and Price Cap
def overload_simulation_with_pricing(station_data, price_cap=3.0):
    """
    Simulate high demand at a few stations and adjust prices accordingly.
    
    Parameters:
    station_data (DataFrame): The DataFrame containing charging station data.
    price_cap (float): The maximum allowed price to prevent unrealistic spikes.
    
    Returns:
    DataFrame: Updated station data with new prices and demand.
    """
    overloaded_stations = station_data.sample(frac=0.1)  # Randomly select 10% of stations
    overload_increase = 15  # Increase kWh total by 15 for overloaded stations
    
    # Increase the kWh total for overloaded stations
    for index, station in overloaded_stations.iterrows():
        station_data.at[index, 'kwhTotal'] += overload_increase  # Simulate increased demand

    # Update prices and calculate new demand based on overload
    for index, station in overloaded_stations.iterrows():
        if station_data.at[index, 'dollars'] > 0:
            # Increase price based on overload situation
            new_price = station_data.at[index, 'dollars'] * (1 + (overload_increase / 100))
            # Cap the price to avoid unrealistic spikes
            new_price = min(new_price, price_cap)
            station_data.at[index, 'dollars'] = new_price
            # Update demand based on the new price using the adjusted demand response
            station_data.at[index, 'new_demand'] = demand_response(new_price)
            print(f"Session {index}: Price {new_price:.2f}, Demand {station_data.at[index, 'new_demand']:.2f}")
        else:
            station_data.at[index, 'new_demand'] = 0  # Set demand to 0 if price is invalid

    return station_data

# Sample station data for testing
# Create a sample DataFrame with random data
data = {
    'sessionId': range(1, 101),
    'kwhTotal': np.random.uniform(0, 10, 100),  # Random kWh totals
    'dollars': np.random.uniform(0.5, 1.5, 100)  # Random prices
}
station_data = pd.DataFrame(data)

# Initialize the new_demand column with zeros to avoid NaN values
station_data['new_demand'] = 0.0

# Run the overload simulation with pricing
updated_station_data = overload_simulation_with_pricing(station_data)

# Display updated data for verification
print(updated_station_data.head())

# Function to plot the prices and demands
def plot_price_vs_demand(station_data):
    """Plot the prices and demands of charging stations."""
    plt.figure(figsize=(12, 6))
    plt.plot(station_data['sessionId'], station_data['dollars'], label='Price ($)', color='blue', marker='o')
    plt.plot(station_data['sessionId'], station_data['new_demand'], label='New Demand', color='orange', marker='x')
    plt.title('Price and Demand of Charging Stations')
    plt.xlabel('Session ID')
    plt.ylabel('Value')
    plt.legend()
    plt.grid()
    plt.show()

# Call the plotting function to visualize the results
plot_price_vs_demand(updated_station_data)
